The code and further work is inspired by: [Liu T., Hsiang B. 2023](https://arxiv.org/pdf/2305.14201.pdf)

# 0. Dataset Generation

Since we do not really have anything to test our theories on, let us create a datset we will use. Obviously, we will create a lot of addition examples, but there is a thing. We are also going to generate pairs of numbers and the result of their *subtraction*, because these two operations are inverse operations, which means addition undoes subtraction and subtraction undoes addition. As a human being that is why I find subtraction examples usefull too. The human logic might not work well with LLM logic, but that is what we are here to try and test.

We will train two different models with and without this part of dataset and compare their results to confirm or disprove my theory. 

The dataset mainly follows Dolly-2.0 style (instruction dataset). It has four keys: 'instruction', 'input', 'output', 'answer'.

*uncomment this if you have not generated the data yet*

In [3]:
# !python3 dataset_generator.py

Addition: 568000
Subtraction: 56800
Total: 624800
Arithmetic dataset generated!
Total: 624800
Dataset generated!


# 1. Prompt Engineering

First, and the most obvious option we have is to choose a small model and set a series of experiments on how a model can work just as "out-of-the-box".

The model we are going to test is [RedPajama-INCITE-3B](https://www.together.xyz/blog/redpajama-3b-updates), because it is an open-source model based on LLaMA, and it it relatively small. It fits perfectly under the constraint of 4B parameters, and also can be rapidly trained with pretty low requirements (and I am a broke student). Its another strength is the tokenisation inherited from LLaMa: it separates the numbers into a set of individual digits, while other models can interpret '232' as '2' and '32' or as '23' and '2' as well (you can see the full comparison within the different models in this work [Nogueira et. al. 2021](https://arxiv.org/pdf/2102.13019.pdf)). Yes, it might be a drawback with something more 'solid', like years, dates, etc. but this is not our case, so it would not affect our performance in that way.

In [ ]:
! pip install transformers
! pip install peft
! pip install datasets

In [ ]:
import re
import sys
import torch
import json
from tqdm.notebook import tqdm
from peft import PeftModel
from sklearn.metrics import accuracy_score
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

from datasets import load_dataset

In [ ]:
TEMPERATURE = 0.1
TOP_P = 0.75
TOP_K = 40
NUM_BEAMS = 1
MAX_NEW_TOKENS = 10

BASE_MODEL = "togethercomputer/RedPajama-INCITE-Instruct-3B-v1"

In [ ]:
from typing import Union


class Prompter(object):
    
    def generate_prompt(
        self,
        instruction: str,
        label: Union[None, str] = None,
    ) -> str:     

        res = f"{instruction}\nAnswer: "
               
        if label:
            res = f"{res}{label}"
         
        return res


    def get_response(self, output: str) -> str:
        return output.split("Answer:")[1].strip()
        # return output

In [ ]:
test_data = load_dataset("json", data_files="data/test.json")

In [ ]:
def inference(
        run_name = 'base',
):
    prompter = Prompter()
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16)
    model = model.to('cuda:0')

    answers = []
    for row in tqdm(test_data["train"]):
        instruction = row["input"]
        prompt = prompter.generate_prompt(instruction)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_length = inputs.input_ids.shape[1]

        with torch.cuda.amp.autocast():
            generation_output = model.generate(
                **inputs, max_new_tokens=5, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id
            )
        output = tokenizer.decode(generation_output.sequences[0])
        answer = prompter.get_response(output)

        try:
            answer = int(answer)
        except:
            answer = int(re.search(r'\d+', output).group())

    
        answers.append({"numbers": row["input"], "answer": answer, "ground_truth": int(row["answer"])})

    with open("./results/" + run_name + ".json", "w") as f:
        json.dump(answers, f, indent=4)

    y_true = [entry['ground_truth'] for entry in answers]
    y_pred = [entry['answer'] for entry in answers]

    accuracy = accuracy_score(y_true, y_pred)
    print('This is how well I have traind my model (accuracy score):', accuracy)
    print('idk why it ended up that bad...')

    #return accuracy

In [ ]:
inference('base') # so, base model can do nothing...

The baseline is basically 0.0. Interesting, let's see what else we can do.

# 2. Fine-Tuning and Test

## 2.1. Training

First of all, we are going to set everything up, and if you want to do some fine-tuning on your own, please run the code below. It was run in colab originally, so if you are using a local machine, you can skip some steps and start generating the data.

In [ ]:
! git clone https://github.com/xufana/4B_LLM_Calculator.git
%cd /content/4B_LLM_Calculator
! pip install -r requirements.txt
! python3 dataset_generator.py --add_volume 100 --sub_volume 100

*uncomment if you want to use wandb*

In [ ]:
#import wandb
#wandb.login()

The default config is 
```
    base_model: str = "togethercomputer/RedPajama-INCITE-Instruct-3B-v1",

    batch_size: int = 16,
    micro_batch_size: int = 4,
    num_epochs: int = 1,
    learning_rate: float = 2e-4,
    cutoff_len: int = 512,
    val_set_size: int = 0,
    
    # lora hyperparams
    lora_r: int = 8,
    lora_alpha: int = 32,
    lora_dropout: float = 0.05,
```

This code took 8 hours to fine-tune on one T4 and used about 10.2Gb VRAM. I used colab pro, but all in all it seems to fit perfectly within the free version. The version with (`batch_size = 128`) and (`micro_batch_size = 16`) took about 12Gb VRAM, so it should work too as well.

Important to notice, I set (`lora_r = 8`), while the authors of the GOAT used (`lora_r = 16`), but I cut it more to fit into colab (I am still a poor student all in all).

In [ ]:
! python3 lora_training.py

`trainable params: 2621440 || all params: 2778485760 || trainable%: 0.09434779323828531`

nice to see

## 2.2. Inference

We will generate some more addition data to test the model:

In [3]:
! python3 dataset_generator.py --dataset_name "test.json" --need_sub False --add_volume 10

Addition: 840
Total: 840
Adding instructions and noise
Total: 840
Dataset generated!


In [ ]:
TEMPERATURE = 0.1
TOP_P = 0.75
TOP_K = 40
NUM_BEAMS = 4
MAX_NEW_TOKENS = 10

Let's load test dataset for the inference:

In [ ]:
import re
import sys
import torch
import json
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

from utils.prompter import Prompter
from datasets import load_dataset

In [ ]:
test_data = load_dataset("json", data_files="data/test.json")

In [ ]:
# MODEL INITIALISATION

BASE_MODEL = "togethercomputer/RedPajama-INCITE-Instruct-3B-v1"
LORA_WEIGHTS = "xufana/RedPajama-3B-Addition"
load_8bit = True

In [ ]:
def inference(
        run_name = 'trained_base',
):
    prompter = Prompter()
    pajama = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL, 
        load_in_8bit=load_8bit,
        torch_dtype=torch.float16,
        device_map='auto',
    )
    model = PeftModel.from_pretrained(
        pajama,
        LORA_WEIGHTS,
        torch_dtype=torch.float16,
        device_map={'': 0},
    )

    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

    answers = []
    for row in tqdm(test_data["train"]):
        instruction = row["input"]
        prompt = prompter.generate_prompt(instruction)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        input_length = inputs.input_ids.shape[1]

        with torch.cuda.amp.autocast():
            generation_output = model.generate(
                **inputs, max_new_tokens=5, do_sample=True, temperature=0.7, top_p=0.7, top_k=50, return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id
            )
        output = tokenizer.decode(generation_output.sequences[0])
        answer = prompter.get_response(output)

        try:
            answer = int(answer)
        except:
            answer = int(re.search(r'\d+', output).group())

    
        answers.append({"numbers": row["input"], "answer": answer, "ground_truth": int(row["answer"])})

    with open("./results/" + run_name + ".json", "w") as f:
        json.dump(answers, f, indent=4)

    y_true = [entry['ground_truth'] for entry in answers]
    y_pred = [entry['answer'] for entry in answers]

    accuracy = accuracy_score(y_true, y_pred)
    print('This is how well I have traind my model (accuracy score):', accuracy)
    print('idk why it ended up that bad...')

    #return accuracy

In [ ]:
inference()

One more 0.0 here...

# 3. Model compression [TBD]

The idea here is to take an existing solution, which is slightly bigger than we need, but we can try and compress it. I will take a model Goat-7B by [Liu T., Hsiang B. 2023](https://arxiv.org/pdf/2305.14201.pdf) ([HF link for the weights](https://huggingface.co/tiedong/goat-lora-7b)). It was already fine-tuned for arithmetics tasks, and especially addition up to 16-digits, so basically it is ready, but just a little bit bigger than required.